<a href="https://colab.research.google.com/github/kzangi/EECS498/blob/main/Copy_of_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CloneExample/
!pwd
!dir


/content/drive/MyDrive/CloneExample
/content/drive/MyDrive/CloneExample
Cloning into 'FavoritePicturesPicasa'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
# print("This script will move the PICASA favorite pictures in a subfolder to a new subfolder in the FavoritePictures folder")
import os
import shutil
from pathlib import Path
import sys
import datetime
import glob

# Define a function to move the favorite pictures to a new directory
def PickFavoritePicturesFcn(dirNAME,cut_off_date):

    path = Path(dirNAME);
    normalized_path = os.path.normpath(dirNAME)   #
    lastDirectory = os.path.basename(normalized_path) # get the last directory in the path
    archDIR = str(path.parent.parent)+"\\"+"FavoritePictures\\"+lastDirectory  # create a new directory to move the pictures to it
    print("Favorite pictures will be moved to directory: ", archDIR)


    # Create a new directory to move the pictures to it
    try:
        os.mkdir(archDIR)
        print("Favorite subdirectory ", archDIR, " created")
    except FileExistsError:
        print("Favorite subdirectory ", archDIR, " already exists")
    except OSError as error:
        print("Error ==>: could not make the favorite subdirectory", error)
        sys.exit(1)



    # copy the *.ini file to the dirName directory
    ini_files = glob.glob(os.path.join(dirNAME, "*.ini")) + glob.glob(os.path.join(dirNAME, ".*.ini"))

    if len(ini_files) == 0:
        print(f'No .ini file was found in {dirNAME}. Will return without copying any files.')
        return(3)

    # get the last time the  *.ini file was modified and compare it with the cut-off date
    ini_file_time = os.path.getmtime(ini_files[0])
    ini_file_time = datetime.datetime.fromtimestamp(ini_file_time)
    if ini_file_time <  cut_off_date:
        print("Will skip the directory ", dirNAME, " because its .ini file is older than the cut-off date")
        return(2)

    # Copy each .ini file to the destination directory
    copyError = False
    for ini_file in ini_files:
        try:
            shutil.copy(ini_file, archDIR)
            print(f'The {os.path.basename(ini_file)} file has been copied to {archDIR}')
            thisINIfile = os.path.basename(ini_file)
            # file_path = archDIR + "\\" + thisINIfile
            fileINI_path = os.path.join(archDIR, thisINIfile)
            lines = []
            # Read the *.ini file and place each line into a list of strings
            with open(fileINI_path, 'r') as f:
                lines = f.readlines()
            print(f'There are {len(lines)} lines in the .picasa file {fileINI_path}')
        except shutil.Error as e:
            print(f"Error copying {os.path.basename(ini_file)} file: {e}")
            return(1)
        except OSError as e:
            print(f"OS error occurred while copying {os.path.basename(ini_file)} file: {e}")
            return(1)



    # Print the lines of the .picasa.ini file
    # print('The lines of the .picasa.ini file are:')
    # for i, line in enumerate(lines, start=1):
    #    print(f'Line number {i}: {line}', end='')


    # Find the linesF with 'star=yes' (favorite pictures) and move the picture file associated with it to the archDIR directory
    iStarPicture = 0
    iCopied      = 0
    i = 0
    for line in lines:
        i += 1
        # print('Line number', i , ': ' ,line, end = '')
        if line == 'star=yes\n' :
            # print('Found start=yes on line number', i)
            iStarPicture = iStarPicture + 1;
            backIndex=1
            while lines[i-backIndex][0] != '[':
                backIndex += 1
            # print('The picture file associated with this star is: ', lines[i-backIndex][1:len(lines[i-backIndex])-2])
            picture_file = lines[i - backIndex][1:len(lines[i - backIndex]) - 2]
            this_picture_Fullfile = os.path.join(dirNAME, picture_file)
            # Copy this picture file to the archDIR directory
            try:
                shutil.copy(this_picture_Fullfile, archDIR)
                iCopied = iCopied + 1
                # print('The picture file', this_picture_Fullfile, 'has been copied to', archDIR)
            except FileNotFoundError as e:
                print(f"File {this_picture_Fullfile} not found error: {e}")
            except shutil.Error as e:
                print(f"Error copying picture file {this_picture_Fullfile}: {e}")
            except OSError as e:
                print(f"OS error occurred while copying picture file {this_picture_Fullfile}: {e}")


            # shutil.copy(dirNAME+"\\"+lines[i-backIndex][1:len(lines[i-backIndex])-2], archDIR)
            # print('The picture file',  lines[i-backIndex][1:len(lines[i-backIndex])-2], ' has been copied to ', archDIR)
    f.close()
    print('There were', iStarPicture, 'favorite pictures in file', fileINI_path, 'and ', iCopied, 'of them were copied to the directory', archDIR)
    # print('DONE \n')
    return(0)
# End of PickFavoritePicturesFcn.py


